### データ正常確認用  

In [ ]:
# coding:utf-8
import json
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import os
import glob
import seaborn as sns
import pickle
import math

In [ ]:
# 解析基準位置に移動する関数
def move_root_dir(root_dir):
    while not (os.path.basename(os.getcwd())== root_dir):
        os.chdir('../')
    print('moving to:',os.path.basename(os.getcwd()))
    return os.getcwd()

# 関数ボックス
def read_csv_file(csv_file):
    df = pd.read_csv(csv_file)
    return df

def plot_confidence_fix(df_data, title, joint_list,head):
    # confidenc
    #title=exp_code+'_'+pp+'_'+ss+'_'+tt+'_head_confidence'
    cols=[]
    offsets=[]
    colors=[]
    
    #joint_list=[29,28,27,30,31]
    joint_name_list = [
            ['pelvis', 0],
            ['naval', 1],
            ['chest', 2],
            ['neck', 3],
            ['l_clavicle', 4],
            ['l_shoulder', 5],
            ['l_elbow', 6],
            ['l_wrist', 7],
            ['l_hand', 8],
            ['l_handtip', 9],
            ['l_thumb', 10],
            ['r_clavicle', 11],
            ['r_shoulder', 12],
            ['r_elbow', 13],
            ['r_wrist', 14],
            ['r_hand', 15],
            ['r_handtip', 16],
            ['r_thumb', 17],
            ['l_hip', 18],
            ['l_knee', 19],
            ['l_ankle', 20],
            ['l_foot', 21],
            ['r_hip', 22],
            ['r_knee', 23],
            ['r_ankle', 24],
            ['r_foot', 25],
            ['head', 26],
            ['nose', 27],
            ['l_eyes', 28],
            ['l_ear', 29],
            ['r_eyes', 30],
            ['r_ear', 31],]
    cm_name = 'jet'# B->G->R
    cm = plt.get_cmap(cm_name,len(joint_list))
    for ic,ii in enumerate(joint_list):
        #cols.append('data_skeletons_0_joints_{}_confidence_0'.format(ii))
        cols.append(head+joint_name_list[ii][0]+'_confidence')
        offsets.append(ic*3)
        colors.append(cm(ic))
    
    df = df_data.copy()
    plt.figure(figsize=(17,5))
    for col, color,offset,joint_no in zip(cols,colors,offsets,joint_list):
        plt.plot(df[col]+offset,label=joint_name_list[joint_no][0],color=color)
        
    plt.title(title)
    plt.legend()
    plt.grid()
    #plt.ylabel('x ')
    #plt.ylim(600,-1700)
    return plt

def get_pallet():
    cmap = plt.get_cmap("tab10")
    cmap = [cmap(0)[0:3],cmap(1)[0:3],cmap(2)[0:3],cmap(3)[0:3],cmap(4)[0:3],cmap(5)[0:3],
            cmap(6)[0:3],cmap(7)[0:3],cmap(8)[0:3],cmap(9)[0:3],cmap(10)[0:3],
            cmap(11)[0:3],cmap(12)[0:3],cmap(13)[0:3],cmap(14)[0:3],cmap(15)[0:3],
           ]
    cbox=[]
    for c in cmap:
        color='#'+format(int(c[0]*255), '02x')+format(int(c[1]*255), '02x')+format(int(c[2]*255), '02x')
        cbox.append(color)
    cmap = cbox
    return cmap

def fill_data(df_data, st, fn):
    df = df_data.copy()
    df['Timestamp'] =pd.to_datetime(df['sensing_time'])
    df = df.set_index('Timestamp')
    df = df.resample('100ms').ffill()
    df = df.reset_index(drop=False)     
    df = df[(df['Timestamp'] > pd.to_datetime(st)) & (df['Timestamp'] <= pd.to_datetime(fn)) ]
    if(len(df)<1):
        return False, df_data        
    df = df.reset_index(drop=False)
    return True, df  


In [ ]:
work_code='20201012_preprocess_data'
root_dir = '2020_moox_morikoro_project'
root_path = move_root_dir(root_dir)
input_data_dir = 'input/20201012_raw_data'
save_csv_dir = 'preprocess_data/'+work_code+'/csvfile'
if not os.path.isdir(save_csv_dir):
    os.makedirs(save_csv_dir)
save_pkl_dir= 'preprocess_data/'+work_code+'/pkl'
if not os.path.isdir(save_pkl_dir):
    os.makedirs(save_pkl_dir)
save_img_dir= 'output/20201012_output_data/fig/'+work_code
if not os.path.isdir(save_img_dir):
    os.makedirs(save_img_dir)

In [ ]:
# 試行リスト
#input file name
input_file_names = glob.glob(input_data_dir+"/20201012_trial_list.xlsx")
print(input_file_names)
# xls book Open (xls, xlsxのどちらでも可能)
input_book = pd.ExcelFile(input_file_names[0]) 
# sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得できる
input_sheet_name = input_book.sheet_names
# lenでシートの総数を確認
num_sheet = len(input_sheet_name)
# シートの数とシートの名前のリストの表示
print ("Sheet の数:", num_sheet)
print (input_sheet_name)
input_sheet_df = input_book.parse(input_sheet_name[0], skiprows = 44)
input_sheet_df = input_sheet_df.iloc[:,1:]
#print(input_sheet_df)
print('reading data')

time_box=[]
for i in range(len(input_sheet_df)):
    box = [
        input_sheet_df.at[i,'num'],
        input_sheet_df.at[i,'p_no'],
        input_sheet_df.at[i,'s_no'],
        input_sheet_df.at[i,'t_no'],
        str(input_sheet_df.at[i,'start_time_UTC']),
        str(input_sheet_df.at[i,'finish_time_UTC']),
        input_sheet_df.at[i,'AzureKinect'],
        input_sheet_df.at[i,'mask'],
        input_sheet_df.at[i,'notation'],        
    ]
    if(box[0]=='num'):
        #print('skip:',box)
        continue
    if(box[0] <= 257):
        #print('skip:',box)
        continue
    if(box[0] == 224 or box[0] == 225):
        #print('skip:',box)
        continue
    time_box.append(box)
    print(box)


### データ読み込み  

In [ ]:
# データセット作成
exp_code = "00_20201012_exp"
data_box=[]
for t_box in time_box:
    if(t_box[1]=='p99'):
        continue
    #if not (t_box[1]=='p28'):
    #    continue
    pp=t_box[1]
    ss=t_box[2]
    tt=t_box[3]
    st=t_box[4]
    fn=t_box[5]
    kinect=t_box[6]
    mask=t_box[7]
    if not (tt=='t202010-92003-450' or tt=='t202010-93003-450' or tt=='t202010-92103-450' or tt=='t202010-93103-450'):
        continue
    
    if(ss=='s02'):
        csv_files=glob.glob(input_data_dir+'/*/detectAction_c_2_parsed_json.csv')
    else:
        csv_files=glob.glob(input_data_dir+'/*/detectAction_c_3_parsed_json.csv')
    for csv_file in csv_files:
        df_data0 = read_csv_file(csv_file)
        is_data,df_data1=fill_data(df_data0, st, fn)
        if not (is_data):
            continue
        df_data=df_data1.copy()
    print(st,fn, len(df_data))
    file_code=save_csv_dir+'/data_df/part_'+pp+ss+tt+'_'+str(mask)+'.csv'
    if not os.path.isdir(save_csv_dir+'/data_df'):
        os.makedirs(save_csv_dir+'/data_df')
    df_data.to_csv(file_code, index=False)
             
    box=[pp,ss,tt,st,fn,df_data,mask]
    data_box.append(box)
print('data_count: ',len(data_box))
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'wb') as web:
    pickle.dump(data_box, web)
print('saved ', finename)

finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)

In [ ]:
# データ読み込み
exp_code = "00_20201012_exp"
finename = save_pkl_dir+'/'+exp_code+'.pkl'
with open(finename, 'rb') as web:
      data_box = pickle.load(web)
      #print techacademy

In [ ]:
# 切り出し時刻チェック
exp_code = "00_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_raw_look_x'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_x'
    x=df[col]
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show()   

In [ ]:
# 切り出し時刻チェック
exp_code = "00_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_look_x'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_x'
    x=df[col]
    if(pp=='p20' and ss=='s02'):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p20' and ss=='s03'):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s02' and is_mask==0):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s02' and is_mask==1):
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s03' and is_mask==0):
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s03' and is_mask==1):
        print(pp,ss,is_mask,len(x))
        #continue
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show()   
    

In [ ]:
# 切り出し時刻チェック
exp_code = "00_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_raw_look_y'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_y'
    x=df[col]
#     if(pp=='p20' and ss=='s02'):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p20' and ss=='s03'):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s02' and is_mask==0):
#         x=list(df[col][300:])
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s02' and is_mask==1):
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s03' and is_mask==0):
#         print(pp,ss,is_mask,len(x))
#         #continue
#     if(pp=='p04' and ss=='s03' and is_mask==1):
#         print(pp,ss,is_mask,len(x))
#         #continue
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show() 

In [ ]:
# 切り出し時刻チェック
exp_code = "00_20201012_exp"
save_dir = save_img_dir+'/'+exp_code
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

title=exp_code+'_marge_look_y'

xmin=0
xmax=4500

plt.figure(figsize=(20,4))
ic=0
for box in data_box:
    pp=box[0]
    ss=box[1]
    tt=box[2]
    st=box[3]
    fn=box[4]
    df_data=box[5]
    is_mask=box[6]
    # データ確認
    df = df_data.copy()
    # 10色パレット
    cmap = get_pallet()
    col='data_fast_status_look_points_look_point_center_y'
    x=df[col]
    if(pp=='p20' and ss=='s02'):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p20' and ss=='s03'):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s02' and is_mask==0):
        x=list(df[col][300:])
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s02' and is_mask==1):
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s03' and is_mask==0):
        print(pp,ss,is_mask,len(x))
        #continue
    if(pp=='p04' and ss=='s03' and is_mask==1):
        print(pp,ss,is_mask,len(x))
        #continue
    
    plt.plot(x,label=pp+ss+tt+'_'+str(is_mask),color=cmap[ic])
    ic=ic+1
#     col='data_fast_status_look_points_look_point_center_y'
#     x2=df[col]
#     plt.plot(x2,label=col,color='G',alpha=0.6)
    
plt.title(title)
plt.ylim(1200,-1200)
plt.xlim(xmin,xmax)
plt.legend(loc="upper left",frameon=False)
# save as png
save_file=title+'.png'
save_path=os.path.join(save_dir, save_file)
plt.savefig(save_path)
plt.show() 

## ズレ調査結果  

p04 s03 0 4500 OK
p04 s02 0 4200 NG
p20 s03 0 4200 NG
p20 s02 0 4200 NG
p04 s03 1 4500 OK
p04 s02 1 4500 OK

- 以下3試行が約30秒ずれている  

p04 s02 0 4200 NG
p20 s03 0 4200 NG
p20 s02 0 4200 NG

切り出しの改善が必要  
切り出しエクセルの時刻を修正...>20201012_trial_list_fit.xlsx